In [144]:

import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import VerletSolver as VS


In [297]:
'''particle class'''

class Planet():
    from Vector import Vector as Vec
    def __init__(self,x = 0,y = 0,z = 0,vx =0,vy = 0,vz = 0,radius=1,color='black'):
        '''Takes in name of the planet, its mass/mass of the sun, and the position and velocity vectors'''

    
        self.x = x
        self.y = y
        self.z = z
        self.pos_lst = [self.x, self.y, self.z]

        self.vx = vx
        self.vy = vy
        self.vz = vz
        self.vel_lst = [self.vx, self.vy, self.vz]
        
        
        self.r = math.sqrt(x*x+y*y+z*z)
        self.v_mag = math.sqrt(vx*vx+vy*vy+vz*vz)
        
        
        self.radius = radius
        self.color = color
        
    def __str__(self):
        return self.name

    
    def grav_accel(self, other, i, scale=1):
        #difference = self.coordinates[coord_index]-other_planet.coordinates[coord_index]
        other
#         sig = 3.405
#         t1 =  (sig** 6)/(math.pow(float(self.distance(other)),7.))
#         t2 = 2(sig**12)/(math.pow(float(self.distance(other)),13.))
#         a = t1-t2
#         return self.pos_lst[i]*scale*a 
    
    def distance(self, other):
        magsqr = 0
        for i in range(len(self.pos_lst)):
            magsqr +=(self.pos_lst[i]-other.pos_lst[i])**2
        return (magsqr)**(0.5)
    
#     def kinetic_energy(self):
#         return .5*self.mass*self.v_mag**2
    
#     def potential_energy(self):
#         try:
#             return -4*math.pi**2*self.mass/self.r
#         except ZeroDivisionError:
#             return 0

#     def angular_momentum(self):
#         return self.mass*self.r*self.v_mag

In [299]:


'''Solar System Class'''

class SolarSystem:
    def __init__(self, iters = 365):
        self.planets = {}
        self.iters = iters
        
    def add_planet(self, inst):
        x_, y_, z_, vx_, vy_, vz_ = np.zeros(self.iters), np.zeros(self.iters), np.zeros(self.iters), np.zeros(self.iters), np.zeros(self.iters), np.zeros(self.iters)
#         ke, pe, te, l = np.zeros(self.iters), np.zeros(self.iters), np.zeros(self.iters), np.zeros(self.iters)
        x_[0] = inst.x
        y_[0] = inst.y
        z_[0] = inst.z
        vx_[0] = inst.vx
        vy_[0] = inst.vy
        vz_[0] = inst.vz

#         ke[0] = planet.kinetic_energy()
#         pe[0] = planet.potential_energy()
#         te[0] = ke[0] + pe[0]
#         l[0] = planet.angular_momentum()
        self.planets[inst] = [x_,y_,z_,vx_,vy_,vz_]
    def __str__(self):
        return_string = "|---BODY---|---X---|---Y---|---Z---|"
        return_string += "\n"

        for p in self.planets:
            return_string += "{:12}{:.6f}  {:.6f} {:.6f} {:.6f}".format(p,p.x,p.y,p.z)
            return_string += "\n"
        return return_string


In [310]:


'''Define Functions'''


def initialize(number_particles):

    length = math.ceil(math.pow(number_particles,1./3.))
    
    length = int(length)
    particle_position=[]
    dict_names = {}
    for x in range(0,length):
        for y in range(0,length):
            for z in range(0,length):
                particle_position.append([x,y,z])
            particle_position.append([x,y,z])
        particle_position.append([x,y,z])
    for i in range(0,length**3):
        v = gen3v(T)
        dict_names[i]=Planet(float(particle_position[i][0]),float(particle_position[i][1]),float(particle_position[i][2]),v[0],v[1],v[2])
    return dict_names    
        
def genv(T,scale):
    '''generate random velocities according to scalable boltzman'''
    v = scale*np.random.normal(0,1)*(T)**.5
    return v

def gen3v(T,scale=1):
    return [genv(T,scale),genv(T,scale),genv(T,scale)]

def kinetic_energy(mass,velocity):
    return .5*mass*velocity**2

def potential_energy(m1, m2, r):
    try:
        return -4*math.pi**2*m1*m2/r
    except ZeroDivisionError:
        return 0

def angular_momentum(m, v, r):
    return m*v*r

def algo(c_i, v_i, r, h):
    a_i = VS.acceleration(c_i, r)
    c_i_1 = VS.coordinate(c_i, h, v_i, a_i)
    a_i_1 = VS.acceleration(c_i_1, r)
    v_i_1 = VS.velocity(v_i, h, a_i_1, a_i)
    return c_i_1, v_i_1, a_i_1


def iterate(SS):
    for i in range(SS.iters-1):
        '''iteration over time steps loop'''
        for p in SS.planets:
            '''planets loop'''
            arrs = SS.planets[p]
            p_lst = []
            v_lst = []
            for j in range(3):
                '''component loop: j = 0 => x & vx; j = 1 => y & vy; j = 2 => z & vz'''
                
                try:
                    c_i = arrs[j][i]
                    v_i = arrs[j+3][i]
                    c_i_1, v_i_1, a_i_1 = algo(c_i, v_i, p.r, h)
                    arrs[j][i+1], arrs[j+3][i+1] = c_i_1, v_i_1
                    p_lst.append(c_i_1)
                    v_lst.append(v_i_1)
                    
                except ZeroDivisionError:
                    p_lst.append(0)
                    v_lst.append(0)
                
            p_vec = Vec(p_lst[0],p_lst[1],p_lst[2])
            v_vec = Vec(v_lst[0],v_lst[1],v_lst[2])
            ke = kinetic_energy(p.mass, v_vec.magnitude())
            pe = potential_energy(p.mass, 1, p_vec.magnitude())
            te = ke + pe
            l = angular_momentum(p.mass, v_vec.magnitude(), p_vec.magnitude())
            arrs[6][i+1], arrs[7][i+1], arrs[8][i+1], arrs[9][i+1] = ke, pe, te, l

def iterate_with_interactions(SS):
    for i in range(SS.iters-1):
        '''iteration over time steps loop'''
        for p in SS.planets:
            '''planets loop'''
            arrs = SS.planets[p]
            p_lst = []
            v_lst = []
            for j in range(3):
                '''component loop: j = 0 => x & vx; j = 1 => y & vy; j = 2 => z & vz'''
                
                try:
                    c_i = arrs[j][i]
                    v_i = arrs[j+3][i]
                    a_i = 0 #VS.acceleration(c_i, p.r)
                    for q in SS.planets:
                        if p != q:
                            #a_i +=
                            print(p.grav_accel(q, j))
                    c_i_1, v_i_1, a_i_1 = algo(c_i, v_i, p.r, h)
                    arrs[j][i+1], arrs[j+3][i+1] = c_i_1, v_i_1
                    p_lst.append(c_i_1)
                    v_lst.append(v_i_1)
                    
                except ZeroDivisionError:
                    p_lst.append(0)
                    v_lst.append(0)
                
            p_vec = Vec(p_lst[0],p_lst[1],p_lst[2])
            v_vec = Vec(v_lst[0],v_lst[1],v_lst[2])
            ke = kinetic_energy(p.mass, v_vec.magnitude())
            pe = potential_energy(p.mass, 1, p_vec.magnitude())
            te = ke + pe
            l = angular_momentum(p.mass, v_vec.magnitude(), p_vec.magnitude())
            arrs[6][i+1], arrs[7][i+1], arrs[8][i+1], arrs[9][i+1] = ke, pe, te, l
                
                
def plot_3D(x,y,z):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(x,y,z,s=5)

    ax.set_xlabel('AU x-axis')
    ax.set_ylabel('AU y-axis')
    ax.set_zlabel('Au z-axis')
    
def plot_3D_pos(dic):
    fig = plt.figure(figsize=(40,22.5))
    ax = fig.add_subplot(111, projection='3d')

    for p in dic:
        ax.plot(xs=dic[p][0],ys=dic[p][1], zs = dic[p][2], c = p.color)
        ax.scatter(xs=dic[p][0][-1],ys=dic[p][1][-1], zs = dic[p][2][-1], s = p.radius*20000, c = p.color)
    ax.set_title('The Solar System')

    ax.set_xlabel('AU x-axis')
    ax.set_ylabel('AU y-axis')
    ax.set_zlabel('AU z-axis')
    
def plot_energies(ke,pe,te,x):
    red_patch = mpatches.Patch(color='red', label='Kinetic Energy')
    blue_patch = mpatches.Patch(color='blue', label='Potential Energy')
    green_patch = mpatches.Patch(color='green', label='Total Energy')

    plt.legend(handles=[red_patch, blue_patch, green_patch],loc=7)
    plt.xlim(0,max(x))
    plt.ylim(min(pe),max(ke)+.1*max(ke))
    plt.xlabel('years')
    plt.ylabel('energy')
    plt.plot(x,ke,color='red')
    plt.plot(x,pe,color='blue')
    plt.plot(x,te,color='green')



In [311]:
T = 10
names =initialize(100)
system  = SolarSystem(10)
for i in range(len(names)): 
#     print( names[i].x,names[i].y, names[i].z , names[i].vx , names[i].vy , names[i].vz)
    system.add_planet( names[i])
print(len(names))

125


In [312]:
iterate_with_interactions(system)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


NameError: global name 'h' is not defined

In [313]:
names[0].pos_list

AttributeError: Planet instance has no attribute 'pos_list'

In [304]:
names

{0: <__main__.Planet instance at 0x7fed835517a0>,
 1: <__main__.Planet instance at 0x7fed83551a28>,
 2: <__main__.Planet instance at 0x7fed835514d0>,
 3: <__main__.Planet instance at 0x7fed835516c8>,
 4: <__main__.Planet instance at 0x7fed83551ef0>,
 5: <__main__.Planet instance at 0x7fed83551830>,
 6: <__main__.Planet instance at 0x7fed83551440>,
 7: <__main__.Planet instance at 0x7fed83551950>,
 8: <__main__.Planet instance at 0x7fed83551518>,
 9: <__main__.Planet instance at 0x7fed83551878>,
 10: <__main__.Planet instance at 0x7fed83551488>,
 11: <__main__.Planet instance at 0x7fed8015a518>,
 12: <__main__.Planet instance at 0x7fed8015abd8>,
 13: <__main__.Planet instance at 0x7fed8354bc68>,
 14: <__main__.Planet instance at 0x7fed8354bd88>,
 15: <__main__.Planet instance at 0x7fed8354bb90>,
 16: <__main__.Planet instance at 0x7fed8354b908>,
 17: <__main__.Planet instance at 0x7fed8354bbd8>,
 18: <__main__.Planet instance at 0x7fed8354bef0>,
 19: <__main__.Planet instance at 0x7fed8